In [19]:
# all imports for this notebook
import json
import requests
import os
import xml.etree.ElementTree as ET
from birdy import WPSClient

In [20]:
# read config file first
with open('config.json', 'r') as f:
    config = json.load(f)
print(config)

{'PEPS_url': 'https://peps.cnes.fr/resto/api/collections/', 'PEPS_collections': ['S1', 'S2', 'S2ST', 'S3'], 'PEPS_formats': ['json', 'atom'], 'COP_url': 'https://scihub.copernicus.eu/dhus/search?q=', 'COP_user': '', 'COP_pwd': '', 'PAVICS_url': 'https://pavics.ouranos.ca/twitcher/ows/proxy/catalog/wps'}


# Connecteur API PEPS

Reference: https://peps.cnes.fr/rocket/plus/img/PEPS-IF-0-0170-ATOS_01_00_[2].pdf
 (pages 8-11)

Examples:
"recherche de produits **S2ST** avec un identifiant **MGRS** égal à **31TCJ** et avec une **couverture nuageuse** comprise entre **10% et 70%**":

https://peps.cnes.fr/resto/api/collections/S2ST/search.json?tileid=31TCJ&cloudCover=[10,70]


La liste des critères de recherche et valeurs disponibles pour chacun des critères est disponible aux adresses
suivantes :
- Les descripteurs Opensearch de la collection [Sentinel 1](https://peps.cnes.fr/resto/api/collections/S1/describe.xml)
- Les descripteurs Opensearch de la collection [Sentinel 2](https://peps.cnes.fr/resto/api/collections/S2/describe.xml)
- Les descripteurs Opensearch de la collection [Sentinel 2 tuilés](https://peps.cnes.fr/resto/api/collections/S2ST/describe.xml)
- Les descripteurs Opensearch de la collection [Sentinel 3](https://peps.cnes.fr/resto/api/collections/S3/describe.xml)


In [6]:
parameters = {
    'tileid' : "31TCJ",
    'cloudCover' : '[10,70]'
}
collection = "S2ST"
out_format = "json"

In [7]:
# construct the search URL
def PEPS_search_url(parameters, collection, out_format):
    search_URL = config['PEPS_url']
    if collection in config['PEPS_collections'] :
      search_URL += collection +"/"
    if out_format in config['PEPS_formats']:
      search_URL += "search." + out_format
    # add any parameters
    for key in parameters:
      search_URL += "&" + key + "=" + str(parameters[key])
    # replace first & with ?
    search_URL = search_URL.replace("&", "?", 1)
    return search_URL

In [8]:
search_PEPS = PEPS_search_url(parameters, collection, out_format)
print(search_PEPS)

https://peps.cnes.fr/resto/api/collections/S2ST/search.json?tileid=31TCJ&cloudCover=[10,70]


Save the query result as a json dictionary

In [10]:
req = requests.get(search_PEPS, allow_redirects=True)
out_peps = req.json()

print("Keys of the json file: ")
for key in out_peps.keys():
    print("\t"+key)
print("Total nr of results: ", out_peps['properties']['totalResults'])
print("Nr of results on this page: ",len(out_peps['features']))
print("First result: ", out_peps['features'][0])

Keys of the json file: 
	type
	properties
	features
Total nr of results:  225
Nr of results on this page:  20
First result:  {'type': 'Feature', 'id': 'c47f5423-1e04-5efa-bd61-7426c46faf26', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[0.53677232313667, 43.238262553327], [1.8886830141923, 43.259391910537], [1.8702372867615, 44.247830683969], [0.49592859290379, 44.225964154763], [0.53677232313667, 43.238262553327]]]]}, '_geometry': {'type': 'MultiPolygon', 'coordinates': [[[[0.53677232313667, 43.238262553327], [1.8886830141923, 43.259391910537], [1.8702372867615, 44.247830683969], [0.49592859290379, 44.225964154763], [0.53677232313667, 43.238262553327]]]]}, 'properties': {'collection': 'S2ST', 'productIdentifier': 'S2A_MSIL1C_20210115T105411_N0209_R051_T31TCJ_20210115T130457', 'parentIdentifier': 'urn:ogc:def:EOP:ESA::SENTINEL-2:', 'title': 'S2A_MSIL1C_20210115T105411_N0209_R051_T31TCJ_20210115T130457', 'description': None, 'organisationName': 'ESA', 'startDate': '2021-01-15T

In [11]:
# get next page's results
def get_next(result):
    for links in result['properties']['links']:
        if links['rel'] == 'self':
            this_page = links['href']
        if links['rel'] == 'next':
            next_page = links['href']
        if links['rel'] == 'last':
            last_page = links['href']
    print("Next: ", next_page)
    if this_page != last_page:
        return next_page
    else :
        print("Last: ", last_page)
        return None
        
get_next(out_peps)

Next:  https://peps.cnes.fr/resto/api/collections/S2ST/search.json?&tileid=31TCJ&cloudCover=%5B10%2C70%5D&page=2


'https://peps.cnes.fr/resto/api/collections/S2ST/search.json?&tileid=31TCJ&cloudCover=%5B10%2C70%5D&page=2'

In [12]:
def get_all_peps_results(url):
    all_results = []
    req = requests.get(url, allow_redirects=True)
    out_peps = req.json()
    for feat in out_peps['features']:
        print(feat['properties']['productIdentifier'])
        all_results += feat
    req_peps = get_next(out_peps)
    while(req_peps):
        req = requests.get(req_peps, allow_redirects=True)
        out_peps = req.json()
        for feat in out_peps['features']:
            print(feat['properties']['productIdentifier'])
            all_results += feat
        req_peps = get_next(out_peps)
    return all_results
        
print(search_PEPS)      
peps_results = get_all_peps_results(search_PEPS)
print("Total results: ", len(peps_results))

https://peps.cnes.fr/resto/api/collections/S2ST/search.json?tileid=31TCJ&cloudCover=[10,70]
S2A_MSIL1C_20210115T105411_N0209_R051_T31TCJ_20210115T130457
S2A_MSIL1C_20210112T104411_N0209_R008_T31TCJ_20210112T125127
S2B_MSIL1C_20210110T105329_N0209_R051_T31TCJ_20210110T130049
S2A_MSIL1C_20201226T105451_N0209_R051_T31TCJ_20201226T130209
S2B_MSIL1C_20201218T104349_N0209_R008_T31TCJ_20201218T115054
S2A_MSIL1C_20201216T105441_N0209_R051_T31TCJ_20201216T130155
S2A_MSIL1C_20201213T104441_N0209_R008_T31TCJ_20201213T125210
S2B_MSIL1C_20201211T105349_N0209_R051_T31TCJ_20201211T130154
S2A_MSIL1C_20201209T110441_N0209_R094_T31TCJ_20201209T131140
S2A_MSIL1C_20201206T105431_N0209_R051_T31TCJ_20201206T130609
S2B_MSIL1C_20201121T105349_N0209_R051_T31TCJ_20201121T115417
S2A_MSIL1C_20201116T105331_N0209_R051_T31TCJ_20201116T130215
S2B_MSIL1C_20201111T105259_N0209_R051_T31TCJ_20201111T130651
S2A_MSIL1C_20201109T110301_N0209_R094_T31TCJ_20201109T131340
S2B_MSIL1C_20201101T105209_N0209_R051_T31TCJ_20201101T

S2A_MSIL1C_20181210T110431_N0207_R094_T31TCJ_20181210T113114
S2A_MSIL2A_20181210T110431_N0211_R094_T31TCJ_20181210T121508
S2A_MSIL2A_20181207T105421_N0211_R051_T31TCJ_20181207T121510
S2A_MSIL1C_20181207T105421_N0207_R051_T31TCJ_20181207T112821
S2B_MSIL2A_20181205T110429_N0211_R094_T31TCJ_20181205T132628
S2B_MSIL1C_20181205T110429_N0207_R094_T31TCJ_20181205T125523
S2A_MSIL1C_20181130T110411_N0207_R094_T31TCJ_20181130T113958
S2A_MSIL2A_20181130T110411_N0211_R094_T31TCJ_20181130T122430
S2B_MSIL2A_20181129T104359_N0211_R008_T31TCJ_20181129T134555
S2B_MSIL1C_20181129T104359_N0207_R008_T31TCJ_20181129T125151
S2A_MSIL2A_20181127T105401_N0211_R051_T31TCJ_20181127T121653
S2B_MSIL2A_20181125T110359_N0211_R094_T31TCJ_20181125T151919
S2A_MSIL2A_20181124T104341_N0211_R008_T31TCJ_20181124T120446
S2A_MSIL1C_20181124T104341_N0207_R008_T31TCJ_20181124T111406
S2B_MSIL1C_20181122T105349_N0207_R051_T31TCJ_20181122T130224
S2B_MSIL2A_20181122T105349_N0211_R051_T31TCJ_20181122T135639
S2B_MSIL1C_20181119T1043

# Connecteur COPERNICUS
Description: https://scihub.copernicus.eu/userguide/OpenSearchAPI

In [21]:
q = "footprint:\"Intersects(41.9000, 12.5000)\""
search_COP = config['COP_url'] + q
search_COP

'https://scihub.copernicus.eu/dhus/search?q=footprint:"Intersects(41.9000, 12.5000)"'

In [23]:
r = requests.get(search_COP, allow_redirects=True, 
                 auth=("samuel.foucher", "2Wsx3edc"))
outfile = 'out_cop.xml'
if r.status_code == 200 and r.content:
    open(outfile, 'wb').write(r.content)
    print(outfile + " written.")
else:
    print("Search request to Copernicus platform unsuccessful. Check credentials!")

out_cop.xml written.


In [24]:
#make sure out_cop.xml is there
if os.path.exists(outfile):
    # parse contents
    tree = ET.parse(outfile)
    root = tree.getroot()
    for child in root:
        print(child.tag, child.attrib, child.text)
        for gchild in child:
              print("\t", gchild.tag, gchild.attrib, gchild.text)

{http://www.w3.org/2005/Atom}title {} Sentinels Scientific Data Hub search results for: footprint:"Intersects(41.9000, 12.5000)"
{http://www.w3.org/2005/Atom}subtitle {} Displaying 0 to 9 of 28681 total results. Request done in 4.626 seconds.
{http://www.w3.org/2005/Atom}updated {} 2021-03-10T19:23:49.183Z
{http://www.w3.org/2005/Atom}author {} 

	 {http://www.w3.org/2005/Atom}name {} Sentinels Scientific Data Hub
{http://www.w3.org/2005/Atom}id {} https://scihub.copernicus.eu/dhus/search?q=footprint:"Intersects(41.9000, 12.5000)"
{http://a9.com/-/spec/opensearch/1.1/}totalResults {} 28681
{http://a9.com/-/spec/opensearch/1.1/}startIndex {} 0
{http://a9.com/-/spec/opensearch/1.1/}itemsPerPage {} 10
{http://a9.com/-/spec/opensearch/1.1/}Query {'role': 'request', 'searchTerms': 'footprint:"Intersects(41.9000, 12.5000)"', 'startPage': '1'} None
{http://www.w3.org/2005/Atom}link {'rel': 'self', 'type': 'application/xml', 'href': 'https://scihub.copernicus.eu/dhus/search?q=footprint:"Inters

# Connecteur PAVICS


In [16]:
wps = WPSClient(config['PAVICS_url'])
help(wps.pavicsearch)

Help on method pavicsearch in module birdy.client.base:

pavicsearch(facets=None, shards='*', offset=0, limit=0, fields='*', format='application/solr+json', query='*', distrib=False, type='Dataset', constraints=None, esgf=False, list_type='opendap_url', output_formats=None) method of birdy.client.base.WPSClient instance
    Search the PAVICS database and return a catalogue of matches.
    
    Parameters
    ----------
    facets : string
        Comma separated list of facets; facets are searchable indexing terms in the database.
    shards : string
        Shards to be queried
    offset : integer
        Where to start in the document count of the database search.
    limit : integer
        Maximum number of documents to return.
    fields : string
        Comme separated list of fields to return.
    format : string
        Output format.
    query : string
        Direct query to the database.
    distrib : boolean
        Distributed query
    type : string
        One of Datase

In [17]:
params = "variable:tasmin,project:CMIP5,experiment:rcp85,frequency:day,institute:CCCma,model:CanESM2"

In [18]:
resp = wps.pavicsearch(constraints=params, limit=10, type="File")
[result, files] = resp.get(asobj=True)
files

['https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/CCCMA/CanESM2/rcp85/day/atmos/r5i1p1/tasmin/tasmin_day_CanESM2_rcp85_r5i1p1_20060101-21001231.nc',
 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/CCCMA/CanESM2/rcp85/day/atmos/r2i1p1/tasmin/tasmin_day_CanESM2_rcp85_r2i1p1_20060101-21001231.nc',
 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/cccma/CanESM2/rcp85/day/atmos/r1i1p1/tasmin/tasmin_day_CanESM2_rcp85_r1i1p1_20060101-21001231.nc',
 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/CCCMA/CanESM2/rcp85/day/atmos/r3i1p1/tasmin/tasmin_day_CanESM2_rcp85_r3i1p1_20060101-21001231.nc',
 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/cccma/CanESM2/rcp85/day/atmos/r5i1p1/tasmin/tasmin_day_CanESM2_rcp85_r5i1p1_20060101-21001231.nc',
 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/CCCMA/CanESM2/rcp85/day/atmos/r4i1p1/tasmin/tasmin_day_CanESM2_rcp85_r4i1p1_200